In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import seaborn as sns
import time
%matplotlib inline

import mdst # my data science tools. A collection of time saving functions on my local machine

In [2]:
#file size causes problems so we will take a random sample of 10,000 rows for this drill
n = 421095 #number of records in file
s = 10000 #desired sample size
skip = sorted(random.sample(range(2, n+2),n-s)) # rows to skip

# load in data passing skip to skiprows kw
y2015 = pd.read_csv(
    'LoanStats3d.csv',
    skipinitialspace=True,
    header=1,
    skiprows=skip
)

# Remove two summary rows at the end that don't actually contain data.
y2015 = y2015[:-2]

c:\python36-32\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (0,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#this cell is copied from curriculum

# Convert ID and Interest Rate to numeric.
y2015['id'] = pd.to_numeric(y2015['id'], errors='coerce')
y2015['int_rate'] = pd.to_numeric(y2015['int_rate'].str.strip('%'), errors='coerce')

# Drop other columns with many unique variables
y2015.drop(['url', 'emp_title', 'zip_code', 'earliest_cr_line', 'revol_util',
            'sub_grade', 'addr_state', 'desc'], 1, inplace=True)

In [4]:
# names of columns that are related to outstanding principal or payment amount
prncp_or_pymnt_amnt = ['out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 
                       'last_pymnt_amnt', 'tot_cur_bal', 'total_bal_il', 'il_util', 'max_bal_bc', 'all_util', 'avg_cur_bal']

# names of columns NOT related to outstanding principal or payment amount
cols = [i for i in y2015.columns if i not in prncp_or_pymnt_amnt]

In [5]:
# set variables
X = y2015[cols].drop('loan_status', 1)
Y = y2015['loan_status']
X = pd.get_dummies(X)
X = X.dropna(axis=1)

In [17]:
from sklearn import ensemble

#time random forest fit 
rfc = ensemble.RandomForestClassifier(max_depth=3)

start_time = time.time()

rfc.fit(X,Y)
end_time = time.time()
print('run time = {}'.format(end_time-start_time))
mdst.display_score(rfc, X, Y)

run time = 0.16590332984924316
accuracy = 0.9047 +/ 0.0209


In [18]:
# time random forest prediction
start_time = time.time()
rfc.predict(X.iloc[3,:].reshape(1,-1))
end_time = time.time()
print('run time = {}'.format(end_time-start_time))


run time = 0.0029969215393066406


c:\python36-32\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  This is separate from the ipykernel package so we can avoid doing imports until


In [19]:
from sklearn import tree

# time decision tree fit
dtc = tree.DecisionTreeClassifier()
dtc.max_depth = 3
start_time = time.time()
dtc.fit(X,Y)
end_time = time.time()
print('run time = {}'.format(end_time-start_time))
mdst.display_score(dtc, X, Y)

run time = 0.19188833236694336
accuracy = 0.9653 +/ 0.0102


In [20]:
# time a prediction
start_time = time.time()
dtc.predict(X.iloc[3,:].reshape(1,-1))
end_time = time.time()
print('run time = {}'.format(end_time-start_time))

run time = 0.0010008811950683594


c:\python36-32\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  This is separate from the ipykernel package so we can avoid doing imports until


the random forest classifier actually fit a bit faster than the decision tree classifier (.17 seconds compared to .19), but the decision tree predicted much faster (.001 seconds vs .003) 